## 2. mongodb 인덱스 (INDEX)
- 검색을 더 빠르게 수행하고자 만든 추가적인 data structure
    - index 데이터 구조가 없다면, 컬렉션에 있는 데이터를 하나씩 조회하는 방식으로 검색이 됨

#### mongodb 접속 기본 코드

In [4]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re # 정규표현식을 위해 

conn = pymongo.MongoClient()
actor_db = conn.cine21
actor_collection = actor_db.actor_collection

### 2.1 기본 인덱스 _id
- 모든 mongodb 컬렉션은 기본적으로 _id 필드를 기반으로 기본 인덱스가 생성됨 

### 2.2 Single(단일) 필드 인덱스

In [18]:
 actor_collection.create_index('배우이름')

'배우이름_1'

In [31]:
actor_collection.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 '직업_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'weights': SON([('직업', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [21]:
docs = actor_collection.find({'배우이름' : '마동석'})
for doc in docs :
    print(doc)

{'_id': ObjectId('622a0fc3dd0d5d6a79a538d5'), '배우이름': '마동석', '흥행지수': 19492, '출연영화': ['나쁜 녀석들: 더 무비', '신과 함께-인과 연', '성난황소', '동네사람들', '원더풀 고스트', '노리개: 그녀의 눈물'], '랭킹': '2', '직업': '배우', '생년월일': '1971-03-01', '성별': '남', '홈페이지': '\nhttps://www.instagram.com/madongseok_/\nhttps://twitter.com/madongseok12\n', '신장/체중': '178cm, 100kg', '다른이름': 'Ma Dongseok'}


#### 부분 문자열 검색을 위해,  text index 생성

In [30]:
 actor_collection.create_index([('직업' ,'text')])

'직업_text'

#### 부분 문자열 검색: $ text

In [32]:
docs = actor_collection.find({'$text' : {'$search' : '가수'}})
for doc in docs :
    print(doc)

{'_id': ObjectId('622a0fc3dd0d5d6a79a53907'), '배우이름': '배수지', '흥행지수': 4531, '출연영화': ['백두산', '건축학개론'], '랭킹': '52', '직업': '가수', '생년월일': '1994-10-10', '성별': '여', '홈페이지': '\nhttps://twitter.com/missa_suzy\nhttps://www.instagram.com/skuukzky/\n', '신장/체중': '166cm, 47kg', '학교': '서울 공연예술고등학교', '소속사': 'JYP엔터테인먼트', '다른이름': '배수지;미쓰에이;'}
{'_id': ObjectId('622a0fc3dd0d5d6a79a538fe'), '배우이름': '임윤아', '흥행지수': 5120, '출연영화': ['기적', '엑시트', '해피 뉴 이어', '공조'], '랭킹': '43', '직업': '가수', '생년월일': '1990-05-30', '성별': '여', '홈페이지': '\nhttps://www.instagram.com/yoona__lim/\n', '소속사': 'SM엔터테인먼트', '다른이름': '소녀시대; girlsgeneration; girls generation'}


- key : ('필드명',direction)
    - direction
        - pymongo.ASCENDING = 1
        - pymongo.DESCENDING = -1
        - pymongo.TEXT ='text'

### 전체 인덱스 삭제

In [35]:
actor_collection.drop_indexes() # 기본 인덱스 이외에는 전부 삭제

In [36]:
actor_collection.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]}}

### 특정 인덱스 삭제

In [11]:
actor_collection.drop_index([(('배우이름', 1))])

In [12]:
actor_collection.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]}}

In [14]:
 actor_collection.create_index([('직업',-1)])

'직업_-1'

In [15]:
actor_collection.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]}, '직업_-1': {'v': 2, 'key': [('직업', -1)]}}

### 2.3 Compound(복합) 필드 인덱스
- 최대 31개의 필드로 만들 수 있음 

In [37]:
actor_collection.create_index([('출연영화',pymongo.TEXT),('직업',pymongo.TEXT)])

'출연영화_text_직업_text'

In [40]:
docs = actor_collection.find({'$text' : {'$search' : '가수'}})
for doc in docs :
    print(doc)

{'_id': ObjectId('622a0fc3dd0d5d6a79a53907'), '배우이름': '배수지', '흥행지수': 4531, '출연영화': ['백두산', '건축학개론'], '랭킹': '52', '직업': '가수', '생년월일': '1994-10-10', '성별': '여', '홈페이지': '\nhttps://twitter.com/missa_suzy\nhttps://www.instagram.com/skuukzky/\n', '신장/체중': '166cm, 47kg', '학교': '서울 공연예술고등학교', '소속사': 'JYP엔터테인먼트', '다른이름': '배수지;미쓰에이;'}
{'_id': ObjectId('622a0fc3dd0d5d6a79a538fe'), '배우이름': '임윤아', '흥행지수': 5120, '출연영화': ['기적', '엑시트', '해피 뉴 이어', '공조'], '랭킹': '43', '직업': '가수', '생년월일': '1990-05-30', '성별': '여', '홈페이지': '\nhttps://www.instagram.com/yoona__lim/\n', '소속사': 'SM엔터테인먼트', '다른이름': '소녀시대; girlsgeneration; girls generation'}


In [39]:
docs = actor_collection.find({'$text' : {'$search' : '신과'}})
for doc in docs :
    print(doc) 

{'_id': ObjectId('622a0fc3dd0d5d6a79a5395b'), '배우이름': '정해균', '흥행지수': 1960, '출연영화': ['신과 함께-인과 연', '나랏말싸미', '성난황소', '신과 함께-죄와 벌', '사라진 시간', '악녀'], '랭킹': '136', '직업': '배우', '생년월일': '1968-08-16', '성별': '남'}
{'_id': ObjectId('622a0fc3dd0d5d6a79a53933'), '배우이름': '이준혁', '흥행지수': 2820, '출연영화': ['신과 함께-인과 연', '야구소녀', '신과 함께-죄와 벌'], '랭킹': '96', '직업': '배우', '생년월일': '1984-03-13', '성별': '남', '홈페이지': '\nhttps://twitter.com/4eyedjack\n', '신장/체중': '181cm, 70kg', '학교': '한신대학교 광고홍보학'}
{'_id': ObjectId('622a0fc3dd0d5d6a79a538fb'), '배우이름': '임원희', '흥행지수': 5418, '출연영화': ['신과 함께-인과 연', '천문: 하늘에 묻는다', '재혼의 기술', '도굴', '죽거나 혹은 나쁘거나', '신과 함께-죄와 벌'], '랭킹': '40', '직업': '배우', '생년월일': '1970-10-11', '성별': '남', '신장/체중': '175cm, 68kg', '학교': '서울예술대학 연극', '소속사': '빌리지엔터테인먼트'}
{'_id': ObjectId('622a0fc3dd0d5d6a79a538f8'), '배우이름': '김동욱', '흥행지수': 5587, '출연영화': ['신과 함께-인과 연', '어쩌다, 결혼', '신과 함께-죄와 벌'], '랭킹': '37', '직업': '배우', '생년월일': '1983-07-29', '성별': '남', '신장/체중': '175cm, 60kg', '학교': '한국예술종합학교 연극원', '취미': '음악듣기, 축구', '특기'

#### 2.4 다양한 문자열 검색

In [41]:
docs = actor_collection.find({'출연영화' : {'$regex' : '엑시트'}})
for doc in docs :
    print(doc) 

{'_id': ObjectId('622a0fc3dd0d5d6a79a538ee'), '배우이름': '강기영', '흥행지수': 7082, '출연영화': ['엑시트', '너의 결혼식', '가장 보통의 연애', '자산어보', '퍼즐'], '랭킹': '27', '직업': '배우', '생년월일': '1983-10-14', '성별': '남', '홈페이지': '\nhttps://instagram.com/kiyoung.kang.5/\n'}
{'_id': ObjectId('622a0fc3dd0d5d6a79a538f1'), '배우이름': '조정석', '흥행지수': 6541, '출연영화': ['마약왕', '뺑반', '엑시트', '형', '시간이탈자', '특종: 량첸살인기'], '랭킹': '30', '직업': '배우', '생년월일': '1980-12-26', '성별': '남', '홈페이지': '\nhttps://twitter.com/jojeol\n'}
{'_id': ObjectId('622a0fc3dd0d5d6a79a538fe'), '배우이름': '임윤아', '흥행지수': 5120, '출연영화': ['기적', '엑시트', '해피 뉴 이어', '공조'], '랭킹': '43', '직업': '가수', '생년월일': '1990-05-30', '성별': '여', '홈페이지': '\nhttps://www.instagram.com/yoona__lim/\n', '소속사': 'SM엔터테인먼트', '다른이름': '소녀시대; girlsgeneration; girls generation'}
{'_id': ObjectId('622a0fc3dd0d5d6a79a5390c'), '배우이름': '박인환', '흥행지수': 4260, '출연영화': ['엑시트', '킹메이커', '봄날은 간다', '푸른 노을', '기묘한 가족', '비밥바룰라'], '랭킹': '57', '직업': '배우', '생년월일': '1945-01-06', '성별': '남', '학교': '중앙대학교 연극영화과 학사'}
{'_id': ObjectId

## <font color = "blue" size = "4em"> 실습 </font><br>
### 배우중에 중앙대학교를 나온 배우를 흥행지수 순으로 최대 10명 출력하라 

In [47]:
docs = actor_collection.find({'학교' : {'$regex' : '경기대학교'}}).sort('흥행지수',pymongo.DESCENDING).limit(10)
for doc in docs :
    print(doc) 

{'_id': ObjectId('622a0fc3dd0d5d6a79a538d9'), '배우이름': '주지훈', '흥행지수': 14826, '출연영화': ['신과 함께-인과 연', '공작', '암수살인', '신과 함께-죄와 벌', '아수라', '서양골동양과자점 앤티크'], '랭킹': '6', '원어명': '주지훈', '직업': '배우', '생년월일': '1982-05-16', '성별': '남', '홈페이지': '\nhttps://www.instagram.com/_jujihoon/\n', '신장/체중': '187cm, 68kg', '학교': '경기대학교 연기', '취미': '컴퓨터 게임', '특기': '속독, 노래, 춤, 합기도, 피아노', '다른이름': '주영훈'}
